<a href="https://colab.research.google.com/github/matheusalvesmartins/API_predictDealsClosings_RealTime_CRM/blob/main/API_PREDI%C3%87%C3%83O_FASTAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn scikit-learn pandas joblib requests

In [ ]:
# Bibliotecas para manipulação de dataset
import pandas as pd
import ast

# Bibliotecas para manipulação de arquivos
from google.colab import files

# Bibliotecas para operações estatísticas e matemáticas
import numpy as np

# Bibliotecas para interações com API
import requests
import time

# Bibliotecas de aprendizado de máquina
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import pipeline

In [ ]:
!pip install fastapi uvicorn transformers



---



In [ ]:
!ngrok config add-authtoken 2zeYUTOZlyoGXtl3hIPYBaC45n7_7R17CPD4oGkFPUvbY8yt8

In [ ]:
# Instalar dependências no Colab (executar uma vez)
!pip install fastapi nest-asyncio pyngrok uvicorn scikit-learn transformers

# Imports
import joblib
import pandas as pd
import requests
import ast
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline
from sklearn.preprocessing import LabelEncoder
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Habilita o FastAPI no Jupyter/Colab
nest_asyncio.apply()

# Carrega o modelo salvo
modelo_pipeline = joblib.load("modelo_predicao_fechamento_ploomes.pkl")

# Define colunas esperadas (exemplo: você pode ajustar)
numerical = ['ValorNegocio']
categorical = ['EstagioAtual', 'StatusAtual', 'MOC', 'OrigemMarketing', 'ERP', 'CRM',
               'OrigemLead', 'Campanha', 'RangeFinanceiro', 'SentimentoCenario', 'Indicacao']

# Pipeline de sentimento
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Funções auxiliares
def analisar_sentimento(texto):
    if pd.isnull(texto) or not isinstance(texto, str) or texto.strip() == "":
        return pd.Series([None, None])
    resultado = sentiment_pipeline(texto[:512])[0]
    return pd.Series([resultado['label'], resultado['score']])

def consultar_dados_brutos_dealid(deal_id, user_key, base_url):
    url = base_url.replace("$ID", str(deal_id))
    headers = {"User-Key": user_key}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return None
    registros = response.json().get("value", [])
    return pd.DataFrame(registros) if registros else None

def preprocessar_dados_dealid(df_raw: pd.DataFrame) -> pd.DataFrame:
    df_raw = df_raw.copy()

    for campo in ["Stage", "Status"]:
        if campo in df_raw.columns:
            df_raw[campo] = df_raw[campo].apply(
                lambda x: ast.literal_eval(x)["Name"] if isinstance(x, str) and x.startswith("{") else (x.get("Name") if isinstance(x, dict) else None)
            )

    if "Contact" in df_raw.columns:
        df_raw["Contact"] = df_raw['Contact'].apply(
            lambda x: x['OtherProperties'][0]['ObjectValueName'] if isinstance(x, dict) and x.get('OtherProperties') else None
        )

    df_main = df_raw.rename(columns={
        "Id": "DealId",
        "ContactId": "Cliente",
        "CreateDate": "DataCriacao",
        "Amount": "ValorNegocio",
        "Stage": "EstagioAtual",
        "Status": "StatusAtual",
        "Contact": "MOC"
    })

    df_main = df_main[["DealId", "Cliente", "DataCriacao", "ValorNegocio", "EstagioAtual", "StatusAtual", "MOC"]]

    df_merge = df_main.copy()
    if "OtherProperties" in df_raw.columns:
        def tentar_avaliar(val):
            if isinstance(val, str) and val.startswith("["):
                try: return ast.literal_eval(val)
                except: return None
            return val

        df_raw["OtherProperties"] = df_raw["OtherProperties"].apply(tentar_avaliar)

        lista_dealids, lista_props = [], []
        for _, row in df_raw.iterrows():
            props = row["OtherProperties"]
            if isinstance(props, list):
                for prop in props:
                    lista_dealids.append(row["Id"])
                    lista_props.append(prop)

        df_other_exploded = pd.DataFrame(lista_props)
        df_other_exploded["DealId"] = lista_dealids
        df_other_exploded["valor_utilizado"] = (
            df_other_exploded.get("ObjectValueName")
            .fillna(df_other_exploded.get("StringValue"))
            .fillna(df_other_exploded.get("BigStringValue"))
        )

        df_other_pivot = df_other_exploded.pivot_table(
            index="DealId", columns="FieldKey", values="valor_utilizado", aggfunc="first"
        ).reset_index()

        df_other_pivot.columns.name = None
        df_other_pivot = df_other_pivot.rename(columns={
            'deal_40906D65-7DCE-4AB1-8D65-F8C142526A41': 'OrigemMarketing',
            'deal_467BB847-34B8-4052-AD55-FB279305CDB7': 'DepartamentoContato',
            'deal_4DAD0A30-82C0-4A91-AC60-9C40B4D8BA31': 'ERP',
            'deal_52AA059C-64B8-45E3-8CCC-127229FCAB38': 'CRM',
            'deal_537A27B9-0DC0-4D12-A975-84E9919598D4': 'OrigemLead',
            'deal_174F0306-11FC-41EA-B575-8794E71CD3CF': 'Cenário',
            'deal_CFBD676E-69E3-408A-9E07-944322B26116': 'Campanha',
            'deal_8B3220DB-2F09-4219-BF7A-65CF2381B9F1': 'ClienteIndicador'
        })

        df_merge = pd.merge(df_merge, df_other_pivot, on="DealId", how="left")

    df_merge["Indicacao"] = df_merge.get("ClienteIndicador").notnull()
    df_merge["RangeFinanceiro"] = pd.cut(df_merge["ValorNegocio"], bins=[0, 2000, 5000, 10000, 20000, float('inf')],
                                         labels=['2k', '5k', '10k', '20k', '20k+'])

    if "Cenário" in df_merge.columns:
        df_merge[["SentimentoCenario", "ScoreSentimento"]] = df_merge["Cenário"].apply(analisar_sentimento)

    col_remover = ['ClienteIndicador', 'DepartamentoContato', 'TamanhoEmpresa', 'Estrutura',
                   'Cliente', 'DataCriacao', 'EstagioAtual', 'Cenário', 'CoreBusiness', 'ScoreSentimento']
    df_merge = df_merge.drop(columns=[c for c in col_remover if c in df_merge.columns])

    for col in numerical + categorical:
        if col not in df_merge.columns:
            df_merge[col] = None

    return df_merge[numerical + categorical]

# --- Criação da API
app = FastAPI(title="API de Predição - Oportunidades Ploomes")

class InputData(BaseModel):
    deal_id: int
    user_key: str
    base_url: str

@app.post("/predict")
def predict(data: InputData):
    df_raw = consultar_dados_brutos_dealid(data.deal_id, data.user_key, data.base_url)
    if df_raw is None:
        return {"erro": "Dados não encontrados para esse DealID."}

    df_tratado = preprocessar_dados_dealid(df_raw)
    proba = modelo_pipeline.predict_proba(df_tratado)[0]
    idx_positiva = list(modelo_pipeline.classes_).index("Ganha")
    prob_ganha = proba[idx_positiva]
    pred = "Ganha" if prob_ganha >= 0.35 else "Perdida"

    return {
        "deal_id": data.deal_id,
        "classe_predita": pred,
        "probabilidades": {"Perdida": float(proba[1-idx_positiva]), "Ganha": float(prob_ganha)}
    }

# --- Executando a API via túnel ngrok
public_url = ngrok.connect(8000)
print(f"🔗 Swagger disponível em: {public_url}/docs")

uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
import requests
from transformers import pipeline
import ast

# --- Carregar modelo treinado
modelo_pipeline = joblib.load("modelo_predicao_fechamento_ploomes.pkl")

# --- Base URL fixa (conforme fornecido)
BASE_URL = "https://api2-s04-app.ploomes.com/Deals?$filter=Id eq $ID+and+true+and+PipelineId+eq+33669&$select=Id,Title,Editable,FirstTaskDate,ContactId,LastQuoteId,LastOrderId,LastDocumentId,StageId,StatusId,PipelineId,Stage,Status,Pipeline,Title,CreateDate,Amount&$expand=Stage($select=Id,Ordination,PipelineId,LastPipelineStage),Status,Owner($select=Id,Name),Creator($select=Id,Name),Pipeline($select=Id,ForbiddenStageReturn,MustPassAllStages,Stages;$expand=Stages),Contact($select=Id,Name,Editable,OwnerId,TypeId),Stage($select=Id,Name,PipelineId,Pipeline,Ordination,LastPipelineStage),Status($select=Id,Name),OtherProperties($expand=CurrencyValue;$filter=FieldId+eq+39587+or+FieldId+eq+68681+or+FieldId+eq+43491+or+FieldId+eq+128142+or+FieldId+eq+89075+or+FieldId+eq+30029262+or+FieldId+eq+56606+or+FieldId+eq+30029253+or+FieldId+eq+27199+or+FieldId+eq+61673+or+FieldId+eq+44134+or+FieldId+eq+30803),Contact($select=Id,OtherProperties;$expand=OtherProperties($filter=FieldId+eq+30008919))&$orderby=CreateDate+desc,Id&preload=true"

# --- Pipeline de sentimento (se necessário)
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# --- Dados de entrada da API
class EntradaAPI(BaseModel):
    deal_id: int
    user_key: str

# --- Função de consulta à API do Ploomes
def consultar_dados_brutos_dealid(deal_id, user_key):
    url = BASE_URL.replace("$ID", str(deal_id))
    headers = {"User-Key": user_key}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return None
    dados = response.json().get("value", [])
    if not dados:
        return None
    return pd.DataFrame(dados)

# --- Pré-processamento (resumo adaptado do seu)
def preprocessar_dados_dealid(df_raw):
    df_raw = df_raw.copy()

    for campo in ["Stage", "Status"]:
        if campo in df_raw.columns:
            df_raw[campo] = df_raw[campo].apply(
                lambda x: ast.literal_eval(x)["Name"] if isinstance(x, str) and x.startswith("{") else (
                    x.get("Name") if isinstance(x, dict) else None
                )
            )

    if "Contact" in df_raw.columns:
        df_raw["Contact"] = df_raw['Contact'].apply(
            lambda x: x['OtherProperties'][0]['ObjectValueName'] if isinstance(x, dict) and x.get('OtherProperties') else None
        )

    df_main = df_raw.rename(columns={
        "Id": "DealId",
        "ContactId": "Cliente",
        "CreateDate": "DataCriacao",
        "Amount": "ValorNegocio",
        "Stage": "EstagioAtual",
        "Status": "StatusAtual",
        "Contact": "MOC"
    })

    df_main = df_main[["DealId", "Cliente", "DataCriacao", "ValorNegocio", "EstagioAtual", "StatusAtual", "MOC"]]

    df_merge = df_main.copy()

    if "OtherProperties" in df_raw.columns:
        def tentar_avaliar(val):
            if isinstance(val, str) and val.startswith("["):
                try:
                    return ast.literal_eval(val)
                except Exception:
                    return None
            return val

        df_raw["OtherProperties"] = df_raw["OtherProperties"].apply(tentar_avaliar)

        lista_dealids = []
        lista_props = []

        for _, row in df_raw.iterrows():
            deal_id = row["Id"]
            props = row["OtherProperties"]
            if isinstance(props, list):
                for prop in props:
                    lista_dealids.append(deal_id)
                    lista_props.append(prop)

        df_other_exploded = pd.DataFrame(lista_props)
        df_other_exploded["DealId"] = lista_dealids

        df_other_exploded["valor_utilizado"] = (
            df_other_exploded.get("ObjectValueName")
            .fillna(df_other_exploded.get("StringValue"))
            .fillna(df_other_exploded.get("BigStringValue"))
        )

        df_other_pivot = df_other_exploded.pivot_table(
            index="DealId",
            columns="FieldKey",
            values="valor_utilizado",
            aggfunc="first"
        ).reset_index()

        df_merge = pd.merge(df_merge, df_other_pivot, on="DealId", how="left")

    df_merge["Indicacao"] = df_merge.get("ClienteIndicador").notnull()
    df_merge["RangeFinanceiro"] = pd.cut(
        df_merge["ValorNegocio"],
        bins=[0, 2000, 5000, 10000, 20000, float('inf')],
        labels=['2k', '5k', '10k', '20k', '20k+']
    )

    if "Cenário" in df_merge.columns:
        df_merge[["SentimentoCenario", "ScoreSentimento"]] = df_merge["Cenário"].apply(analisar_sentimento)

    col_remover = [
        'ClienteIndicador', 'DepartamentoContato', 'TamanhoEmpresa',
        'Estrutura', 'Cliente', 'DataCriacao',
        'EstagioAtual', 'Cenário', 'CoreBusiness', 'ScoreSentimento'
    ]
    df_merge = df_merge.drop(columns=[c for c in col_remover if c in df_merge.columns], errors="ignore")

    colunas_esperadas = modelo_pipeline.feature_names_in_
    for col in colunas_esperadas:
        if col not in df_merge.columns:
            df_merge[col] = None
    df_merge = df_merge[colunas_esperadas]

    return df_merge

# --- Instância da API FastAPI
app = FastAPI()

@app.post("/predict")
def predict_entrada(dados: EntradaAPI):
    df_raw = consultar_dados_brutos_dealid(dados.deal_id, dados.user_key)

    if df_raw is None:
        return {"erro": "Não foi possível obter os dados para o DealId informado."}

    df_tratado = preprocessar_dados_dealid(df_raw)

    probas = modelo_pipeline.predict_proba(df_tratado)[0]
    pos_idx = list(modelo_pipeline.classes_).index("Ganha")
    threshold = 0.35
    classe = "Ganha" if probas[pos_idx] >= threshold else "Perdida"

    return {
        "DealId": dados.deal_id,
        "Classe": classe,
        "Probabilidades": {
            "Perdida": round(probas[1 - pos_idx], 3),
            "Ganha": round(probas[pos_idx], 3)
        }
    }


In [ ]:
!pip install fastapi pyngrok uvicorn nest_asyncio

In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"🔗 Acesse a interface da API via Swagger: {public_url}/docs")

uvicorn.run(app, port=8000)
